# Let's Create a Simple Blockchain

At the very core, we want to create a linked list. Each block points to the fingerprint of the previous block. Additionally, we include a checksum of the content of each block (i.e., fingerprint).

<img src="include/block_diagram.png" alt="block diagram/" style="width: 300px;"/>

## *<font color=" #6495ED">Exercise</font>*

 - Define a class called "block" to represent the structure above

In [ ]:
# SOLUTION


In [ ]:
test_block = block(b"1", b"DUMMY DATA")

In [ ]:
test_block

## Genesis Block 

As we can now we can represent each block, and we can point to the previous block. Any modification to any block will propagate to the other blocks. If the data in any block changes, the hash changes, and this would reflect in the consequent blocks.

BUT, where does the first block come from? There is no previous block to the first block.

We should define one block as the **genesis block** where every other block agrees upon.

## *<font color=" #6495ED">Exercise</font>*

 - Define your genesis block, with following parameters:
     - previous hash: "0000000000000000000000000000000000000000000000000000000000000000
     - nonce: b"FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF"
     - data: "PyCon 2018 Genesis Block"

In [ ]:
# SOLUTION


## Putting the chain into blockchain

In our simple blockchain, we will use a *list* to store each block of our blockchain.

## *<font color=" #6495ED">Exercise</font>*
 - Define a class called "blockchain", define a the following functions
     - "\__init__": initialize a empty list
     - "add_blocks": simply append a block to the list, (block is passed as an argument)
     - "blockchain_valid": to check if the blockchain is valid, for now just return True

In [ ]:
# SOLUTION


In [ ]:
test_blockchain = blockchain()

In [ ]:
test_blockchain

In [ ]:
test_blockchain.blocks

In [ ]:
test_blockchain.add_block(genesis_block)

In [ ]:
test_blockchain

In [ ]:
test_blockchain.blocks

## Blockchain validity
Now we want to check if our blockchain is valid. In our example, we want to check if the hash of each block is correct. First we need to create a hash of the block header + data and check if it is correct. Furthermore, we need to check if the previous hash in each block is correct.

## *<font color=" #6495ED">Exercise</font>*
 - Implement the "blockchain_valid", according to the description

In [ ]:
# SOLUTION



In [ ]:
# Good Genesis Block
test_blockchain = blockchain()
test_blockchain.add_block(genesis_block)
print(test_blockchain.blockchain_valid()) # this should be True

In [ ]:
# Bad Genesis Block
test_blockchain = blockchain()
test_blockchain.add_block(block(b"1", b"2"))
print(test_blockchain.blockchain_valid(), "--Bad Genesis Block--") # this should be False

In [ ]:
# Blocks are correct
test_blockchain = blockchain()
test_blockchain.add_block(genesis_block)
print(test_blockchain.blockchain_valid())

print(genesis_block)
test_block = block(genesis_block.hash, b"DUMMY DATA")
print(test_block)
test_blockchain.add_block(test_block)
test_blockchain.blockchain_valid()

In [ ]:
test_block2 = block(test_block.hash, b"DUMMY DATA2")
print(test_block2)
test_blockchain.add_block(test_block2)
test_blockchain.blockchain_valid()

In [ ]:
print(test_blockchain)

## Proof of Work

Well, now we have a basic blockchain working. However, there is one small problem, everyone can create blocks and chain them together. There is not much effort needed to create new blocks, hash them and add them to the ***list***.

### Hashing to the rescue

Why not make the creation of valid blocks hard. Then, we only accepts the blocks that have certain amount of work and effort done on them as valid. Each block hash should start with certain amount of zeros. We have set the number of zeros dynamically to be adapt with new technology and also number of ***block truthfulness seekers*** (miners).

## *<font color=" #6495ED">Exercise</font>*
 - Let's put this in code. Set the number of leading zeros to 3.

In [ ]:
# SOLUTION


In [ ]:
# Blocks are correct
test_blockchain = blockchain()
test_blockchain.add_block(genesis_block)

In [ ]:
test_blockchain

In [ ]:
test_block = block(genesis_block.hash, b"DUMMY DATA")
test_block

In [ ]:
test_block.find_nonce(3)
test_block

In [ ]:
test_blockchain.add_block(test_block)
test_blockchain.blockchain_valid()

In [ ]:
test_block2 = block(test_block.hash, b"DUMMY DATA2")
print(test_block2, "\n")
test_block2.find_nonce(3)
print(test_block2)
test_blockchain.add_block(test_block2)
test_blockchain.blockchain_valid()

In [ ]:
test_blockchain

## The tale of two blockchains

Once again, we have something working. But there is one more problem, if you have to ***branches*** of the blockchain that are ***valid***, how to decide which one to use. We only want to have one true blockchain branch.

Why not go with the blockchain that has the most effort and work done on it then? This translates into selecting the blockchain with the longest chain length. Easy!

In [ ]:
test_blockchain1 = blockchain()
test_blockchain2 = blockchain()

test_blockchain1.add_block(genesis_block)
test_blockchain2.add_block(genesis_block)


print(test_blockchain1)
print()
print(test_blockchain2)

## *<font color=" #6495ED">Exercise</font>*
 - Set the difficulty to 3 and start creating two branches of the blockchain for 10 second each.

In [ ]:
# SOLUTION


In [ ]:
mine_blocks(test_blockchain1)

In [ ]:
mine_blocks(test_blockchain2)

In [ ]:
len(test_blockchain1.blocks)

In [ ]:
len(test_blockchain2.blocks)

Congratulations! You just created a simple blockchain, with mining, dynamic difficulty level, proof of work, and longest chain consensus algorithm. Now, what does it take to make our blockchain into a cryptocurrency?

## Cryptocurrency

To transfer our correct blockchain into a cryptocurrency, instead of storing random data, we need to store transaction data.

 - From: Alice
 - To: Bob
 - Amount: 10$

## *<font color=" #6495ED">Exercise</font>*
 - If we just store the transactions, what stops anyone to put wrong transactions there?
 - Bob can just create a message indicating Alice-> Bob 100000..0000$

## Aysmmetric Encryption System and Signatures

Fortunately, it's not too hard to mitigate against this.

 - Instead of Alice, Bob, X let's use $H(pub(Alice))$, $H(pub(Bob))$, $H(pub(X))$. $ADDR_X=H(pub(X))$
 - Now just sign the hash of a transaction with your private key
 - $S_X(H(ADDR_X -> ADDR_Y: 10\$))$, S_X, signing using private key of X
 

In [ ]:
#TX data from, to, amount 
tx_data = "{0}======={1}======={2}"

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.serialization import PublicFormat
from cryptography.hazmat.primitives.serialization import Encoding
from cryptography.hazmat.primitives import serialization
import base64

def hash_pub_key(private_key):
        digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
        digest.update(private_key.public_key().public_bytes(Encoding.DER, PublicFormat.SubjectPublicKeyInfo))
        return base64.b16encode(digest.finalize())

def sign_tx(tx, private_key):
    return private_key.sign(tx, ec.ECDSA(hashes.SHA256()))

def serialize_pubkey(publickey):
    serialized_public = publickey.public_bytes(
    encoding=Encoding.PEM,
    format=PublicFormat.SubjectPublicKeyInfo)
    
    return serialized_public

def parse_serialized_pubkey(serialized_pubkey):
    loaded_public_key = serialization.load_pem_public_key(serialized_pubkey, backend=default_backend())
    return loaded_public_key

## *<font color=" #6495ED">Exercise</font>*
 - Define three private keys (Alice, Bob, Carol) and H(pubkey) for each 

In [ ]:
# SOLUTION    


## Creating a transaction + signature

In [ ]:
# Alice -> Bob, 10$
tx_data_b = str.encode(tx_data.format(hash_pub_key(private_keys["Alice"]),
                          hash_pub_key(private_keys["Bob"]),
                          "10"))
serialized_pubkey = serialize_pubkey(private_keys["Alice"].public_key())
siganture = sign_tx(tx_data_b, private_keys["Alice"])

In [ ]:
# transaction data
tx_b = tx_data_b+b'======='+serialized_pubkey+b'======='+siganture

In [ ]:
# Bob -> Carol, 15$
tx_data_b = str.encode(tx_data.format(hash_pub_key(private_keys["Bob"]),
                          hash_pub_key(private_keys["Carol"]),
                          "15"))
serialized_pubkey = serialize_pubkey(private_keys["Bob"].public_key())
siganture = sign_tx(tx_data_b, private_keys["Bob"])

In [ ]:
# transaction data
tx_b2 = tx_data_b+b'======='+serialized_pubkey+b'======='+siganture

In [ ]:
# Carol -> Alice, 20$
tx_data_b = str.encode(tx_data.format(hash_pub_key(private_keys["Carol"]),
                          hash_pub_key(private_keys["Alice"]),
                          "20"))
serialized_pubkey = serialize_pubkey(private_keys["Carol"].public_key())
siganture = sign_tx(tx_data_b, private_keys["Carol"])

In [ ]:
# transaction data
tx_b3 = tx_data_b+b'======='+serialized_pubkey+b'======='+siganture

## *<font color=" #6495ED">Exercise</font>*
 - Let's add 1 transactions to each block
 - Check if the signature is correct

In [ ]:
# SOLUTION


In [ ]:
# Blocks are correct
test_blockchain = blockchain()
test_blockchain.add_block(genesis_block)

In [ ]:
test_block = block(genesis_block.hash, tx_b)
test_block.find_nonce(3)
test_blockchain.add_block(test_block)

test_block = block(test_block.hash, tx_b2)
test_block.find_nonce(3)
test_blockchain.add_block(test_block)

test_block = block(test_block.hash, tx_b3)
test_block.find_nonce(3)
test_blockchain.add_block(test_block)

test_blockchain.blockchain_valid()

In [ ]:
test_blockchain

In [ ]:
for _block in test_blockchain.blocks[1:]:
    verify_tx_signatures(_block.data)

## *<font color=" #6495ED">Exercise</font>*
 - How to make sure the money is not already spent? (double spending)